In [69]:
from mongoengine import *
import numpy as np
import datetime 
import pandas
from sklearn import *

In [70]:
connect("GDP-test", host = "localhost", port = 27017)
class Detection(Document):
    id_webcam = IntField(required=True)
    city = StringField(required=True)
    location = StringField(required=True)
    latitude = FloatField(required=True)
    longitude = FloatField(required=True)
    numPeople = IntField(required=True)
    date = DateTimeField(required=True)
    time = DateTimeField(required=True)
    type = IntField(required=True)
    weather_description = StringField()
    temperature = FloatField()
    day_of_week = IntField()

In [112]:
print(Detection.objects.distinct('city')) # vedere se posso prendere solo le città per poi fare query distinte
table = pandas.DataFrame(Detection.objects(city='Roma').as_pymongo()) #prelevo i dati di roma
table = table.drop(columns = ['_id','id_webcam','city','date','type','location','latitude','longitude']) #tolgo colonne inutili
table['time'] = table['time'].dt.strftime('%H').astype(int)#modifico le colonne con i valori che mi servono
print(table)
table['weather_description'] = pandas.get_dummies(table['weather_description']) #il weather viene convertito con parametri interi




['Roma', 'Novska', 'Djakovo', 'Krk']
      numPeople  time weather_description  temperature  day_of_week
0             6     9           clear sky         7.10            2
1             6     9           clear sky         7.10            2
2             7     9           clear sky         7.10            2
3            10     9           clear sky         8.25            2
4             6     9           clear sky         8.25            2
...         ...   ...                 ...          ...          ...
1411          8     8           clear sky         8.18            0
1412          8     8           clear sky         8.18            0
1413          6     8           clear sky         9.44            0
1414         11     8           clear sky         9.44            0
1415          8     8           clear sky         9.44            0

[1416 rows x 5 columns]


In [116]:
print(table[['time','weather_description','temperature','day_of_week']])
model = sklearn.linear_model.LinearRegression()
model.fit(table[['time','weather_description','temperature','day_of_week']],table['numPeople']) #provo a dargli in pasto tutto
#prima di questo ora bisogna dividere tutto il dataset in 70-30

      time  weather_description  temperature  day_of_week
0        9                    0         7.10            2
1        9                    0         7.10            2
2        9                    0         7.10            2
3        9                    0         8.25            2
4        9                    0         8.25            2
...    ...                  ...          ...          ...
1411     8                    0         8.18            0
1412     8                    0         8.18            0
1413     8                    0         9.44            0
1414     8                    0         9.44            0
1415     8                    0         9.44            0

[1416 rows x 4 columns]


LinearRegression()

In [87]:

#robe vecchie sbagliate non guardare

time, temperature, weather, numP = table['time'].dt.strftime('%H').astype(int),table['temperature'],table['weather_description'],table['numPeople']
time = np.asarray(time)
temperature = np.asarray(temperature)
weather = np.asarray(weather)
numP = np.asarray(numP)
weatherDict = {'clear sky':0}

In [60]:
for i in range(weather.size):
    weather[i] = weatherDict[weather[i]]

In [61]:
temperature = temperature.reshape(-1,1)
time = time.reshape(-1,1)
weather = weather.reshape(-1,1)
timeTemp = np.concatenate((time,temperature,weather),axis = 1)
print(timeTemp)
model = sklearn.linear_model.LinearRegression()
model.fit(timeTemp,numP) 

[[11 13.48 0]
 [11 13.47 0]
 [11 13.47 0]
 [11 13.47 0]
 [11 13.47 0]
 [11 13.47 0]
 [11 13.47 0]
 [11 13.47 0]]


LinearRegression()

Coefficients: 
 [    0.         -1057.14285714     0.        ]
